In [15]:
import json
import itertools
from datetime import date

import cv2
import rasterio
import utm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub import pixel_to_utm, utm_to_pixel

In [16]:
geojson_path = "../data/cameroon.geojson"

with open("../secrets/sentinel_api_credentials.json", "r") as file:
    credentials = json.load(file)

api = SentinelAPI(credentials["username"], credentials["password"], "https://scihub.copernicus.eu/dhus")

shape = geojson_to_wkt(read_geojson(geojson_path))

images = api.query(
    shape,
    date=(date(2020, 5, 1), date(2020, 5, 10)),
    platformname="Sentinel-2",
    processinglevel = "Level-2A",
    cloudcoverpercentage=(0, 30)
)

images_df = api.to_dataframe(images)

The query string is too long and will likely cause a bad DHuS response.


ServerError: HTTP status 400 Bad request: 
# 400 Bad request

Your browser sent an invalid request.